# Supercritical Steam Cycle Example

This example uses Jupyter Lab, and demonstrates a supercritical power plant steam cycle.  See the ```supercritical_steam_cycle.py``` file run in the next cell for the full flowsheet set up example.  To run this example without modification, launch JupyterLab from the example directory. You can view the Python code or generated SVG files by double clicking them in the left sidebar.

## Initialize the steam cycle flowsheet

Running the ```supercritical_steam_cycle.py``` script will create the steam cycle model in an object called ```m``` and Pyomo solver object called ```solver```.  The results of the simulation are displayed in the ```results.svg``` file, which can be viewed by selecting it from the left sidebar.

In [1]:
%run supercritical_steam_cycle.py

2019-08-19 13:33:09 - INFO - idaes.unit_models.separator - fs.turb.inlet_split Initialisation Complete.
2019-08-19 13:33:09 - INFO - idaes.core.unit_model - fs.turb.throttle_valve[1] Initialisation Step 1 Complete.
2019-08-19 13:33:09 - INFO - idaes.core.unit_model - fs.turb.throttle_valve[1] Initialisation Step 2 Complete.
2019-08-19 13:33:09 - INFO - idaes.core.unit_model - fs.turb.throttle_valve[1] Initialisation Complete.
2019-08-19 13:33:09 - INFO - idaes.core.unit_model - fs.turb.throttle_valve[2] Initialisation Step 1 Complete.
2019-08-19 13:33:09 - INFO - idaes.core.unit_model - fs.turb.throttle_valve[2] Initialisation Step 2 Complete.
2019-08-19 13:33:09 - INFO - idaes.core.unit_model - fs.turb.throttle_valve[2] Initialisation Complete.
2019-08-19 13:33:09 - INFO - idaes.core.unit_model - fs.turb.throttle_valve[3] Initialisation Step 1 Complete.
2019-08-19 13:33:09 - INFO - idaes.core.unit_model - fs.turb.throttle_valve[3] Initialisation Step 2 Complete.
2019-08-19 13:33:09 - 

For this example, we will see the effect of partially closing one of the throttle valves.  The gross power output from the turbine is shown below.  The displayed value is in Watts, and the negative sign indicates power is being produced.

In [2]:
pyo.value(m.fs.turb.power[0])

-610029687.4881779

## Change the model inputs

The turbine in this example simulates partial arc admission with four arcs, so there are four throttle valves.  For this, example we will close one of the valves to 20% open, and observe the result.

In [3]:
m.fs.turb.throttle_valve[1].valve_opening[:].value = 0.2

Next we re-solve the model using the solver created by the ```supercritical_steam_cycle.py``` script.

In [4]:
solver.solve(m, tee=True)

Ipopt 3.12.13: tol=1e-06
max_iter=300


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:     3594
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:     1724

Total number of variables............................:     1012
                     variables with only lower bounds:      259
                variables with lower and upper bounds:      512
                     variables with only upper bounds:        0
Total number of equality constraints.................:     

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 1012, 'Number of variables': 1012, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.12.13\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 20.738149881362915}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

Now we can check the gross power output again.

In [5]:
pyo.value(m.fs.turb.power[0])

-559558446.2160016

## Creating a PFD with results and a stream table

Next we can take a closer look at the model results. For this model there are two convenient ways to see the results.  One is to generate a stream table as a Pandas data frame.  The other is to use the stream table to generate a PFD with the streams and other results labeled.

The cell below creates a stream table, then uses a function defined in ```supercritical_steam_cycle.py``` to generate a PFD with results.

In [6]:
df = create_stream_table_dataframe(streams=m._streams, orient="index")
pfd_result("result.svg", m, df)

The PFD (```results.svg```) can be viewed by selecting it from the left sidebar.  The cell below displays the stream table.

In [7]:
df

,Molar Flow (mol/s),Mass Flow (kg/s),T (K),P (Pa),Vapor Fraction,Molar Enthalpy (J/mol) Vap,Molar Enthalpy (J/mol) Liq
COND_01,1.802654e+04,324.753243,302.504773,4.092180e+03,1.878855e-11,4.601788e+04,2216.540380
COND_02,1.802654e+04,324.753243,302.504773,4.092180e+03,1.878515e-11,4.601788e+04,2216.540380
COND_03,1.802654e+04,324.753243,302.586462,1.004092e+06,0.000000e+00,-7.003515e+07,2239.144952
CW01,2.000000e+06,36030.560000,294.209445,5.000000e+05,0.000000e+00,-2.720303e+04,1600.000000
CW02,2.000000e+06,36030.560000,299.504773,5.000000e+05,0.000000e+00,-8.108357e+07,1998.832628
EXHST_BFPT,4.086848e+03,73.625719,437.206256,6.428605e+04,1.000000e+00,5.056354e+04,12480.451872
EXHST_MAIN,1.393970e+04,251.127524,302.504773,4.092180e+03,9.828048e-01,4.601788e+04,2216.540380
EXTR_BFPT_A,4.086848e+03,73.625719,536.626895,2.571442e+05,1.000000e+00,5.398413e+04,20782.703310
EXTR_HP4,1.510148e+03,27.205746,659.110742,6.025051e+06,1.000000e+00,5.659215e+04,56592.151192
EXTR_HP7,1.164395e+03,20.976909,573.361520,3.084826e+06,1.000000e+00,5.390572e+04,24397.039273


In [10]:
m.fs.turb.throttle_valve[1].valve_opening.display()

valve_opening : Fraction open for valve from 0 to 1
    Size=1, Index=fs.time
    Key : Lower : Value : Upper : Fixed : Stale : Domain
    0.0 :     0 :   0.2 :     1 :  True :  True :  Reals


In [11]:
m.fs.turb.throttle_valve[1].valve_opening.unfix()